In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from waffles.np02_utils.PlotUtils import matplotlib_plot_WaveformSetGrid
from waffles.np02_utils.AutoMap import ordered_modules_pmt, ordered_channels_pmt, dict_uniqch_to_module, strUch
from waffles.data_classes.EasyWaveformCreator import EasyWaveformCreator
from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.WaveformSet import WaveformSet


In [ ]:
url = "https://docs.google.com/spreadsheets/d/1GZeYsvjF3_33yhto1ivWYGZeV0QHzG7hhyoyaFT88r4/export?format=xlsx"
df = pd.read_excel(url, sheet_name="Foglio1")
df = df.dropna()

In [ ]:
df = df[~df['CHANNEL'].isin([42, 44])]
channels = df['CHANNEL'].unique()

In [ ]:
dict_channels = { 110: [ ch for ch in ordered_channels_pmt if ch not in [42, 44]] }
detectors = [ UniqueChannel(110,ch) for ch in ordered_channels_pmt if ch not in [42, 44]] 
 
wfset = EasyWaveformCreator.create_WaveformSet_dictEndpointCh(dict_channels)

In [ ]:
ordered_channels_pmt.sort()
ordered_channels_pmt

In [ ]:
def pmt_gains_plot(waveform:Waveform, df, fit_results):
    channel = waveform.waveforms[0].channel
    channel_data = df[df['CHANNEL'] == channel].sort_values('Voltage')
    
    # Extract ALL data for plotting
    voltage_all = channel_data['Voltage'].values
    gain_all = channel_data['Gain'].values
    err_gain_all = channel_data['Err_Gain'].values
    
    # Use subset for fitting (exclude last point)
    voltage = voltage_all[:-1]
    gain = gain_all[:-1]
    err_gain = err_gain_all[:-1]
    
    # Linear fit in log space: log(Gain) = a * Voltage + b
    log_gain = np.log10(gain)
    coeffs = np.polyfit(voltage, log_gain, 1)  # Linear fit
    a, b = coeffs[0], coeffs[1]
    
    # Find voltage where Gain = 2*10^7
    target_gain = 2*1e7
    target_voltage = (np.log10(target_gain) - b) / a
    fit_results.append((channel, target_voltage))
    
    # Create fit line for plotting
    voltage_fit = np.linspace(voltage.min() - 20 , voltage.max() + 20, 100)
    gain_fit = 10**(a * voltage_fit + b)

    # Plot
    plt.errorbar(voltage_all, gain_all, yerr=err_gain_all, 
                 marker='o', linestyle='', capsize=2, label='Data')
    plt.plot(voltage_fit, gain_fit, 'r-', label='Fit')
    plt.axhline(y=2e7, color='g', linestyle='--', label=f'Gain = 2*10^7') 
    plt.axvline(x=target_voltage, color='g', linestyle='--', 
                label=f'V = {target_voltage:.1f}')
    plt.yscale('log')
    plt.xlabel('Voltage [V]')
    plt.ylabel('Gain')
    plt.legend(title=f'{dict_uniqch_to_module[strUch(110, channel)]}: 110-{channel}')
    
    print(f"Channel {channel}: Voltage at Gain = 2*10^7 is {target_voltage:.2f} V")

In [ ]:
fit_results = []

params_plot = dict(
    df = df,
    fit_results = fit_results,
)

pmtuch = dict_channels[110]

#detectors = [ UniqueChannel(110,ch) for ch in ordered_channels_pmt if ch not in [42, 44]] 
detectors=[ UniqueChannel(110, ch) for ch in pmtuch[:len(pmtuch)//2] ]
matplotlib_plot_WaveformSetGrid(wfset, detector=detectors, plot_function=pmt_gains_plot, func_params=params_plot, cols=2, figsize=(16,25))
plt.savefig('fit_results.pdf')
detectors=[ UniqueChannel(110, ch) for ch in pmtuch[len(pmtuch)//2:] ]
matplotlib_plot_WaveformSetGrid(wfset, detector=detectors, plot_function=pmt_gains_plot, func_params=params_plot, cols=2, figsize=(16,25))
plt.savefig('fit_results2.pdf')

In [ ]:
print("Channel, Voltage at Gain=2*10^7")
for channel, voltage in fit_results:
    print(f"{channel}, {voltage:.0f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define marker styles and colors
markers = ['o', 's', '^', 'v', 'D', '*', 'p', 'h', '<', '>', 'X', 'd']
colors = plt.cm.tab10(range(10))  # Use matplotlib's color cycle

plt.figure(figsize=(10, 8))

for idx, channel in enumerate(channels):
    channel_data = df[df['CHANNEL'] == channel].sort_values('Voltage')
    
    # Extract data
    voltage = channel_data['Voltage'].values
    gain = channel_data['Gain'].values
    err_gain = channel_data['Err_Gain'].values
    
    if len(voltage) < 2:
        continue
    
    # Plot with unique marker and color
    marker = markers[idx % len(markers)]
    color = colors[idx % len(colors)]
    
    plt.errorbar(voltage, gain, yerr=err_gain, 
                 marker=marker, linestyle='--', capsize=2, 
                 color=color, label=f'Channel {channel}')

plt.axhline(y=1e7, color='black', linestyle=(0,(5,10)), linewidth=1, label='Gain = 10^7')
plt.yscale('log')
plt.title('Gain vs Voltage - All Channels')
plt.xlabel('Voltage (V)')
plt.ylabel('Gain')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()